In [4]:
import numpy as np

class Board():
    def __init__(self):
        self.state = np.zeros((6, 7))  # 6 filas y 7 columnas

    def valid_moves(self):
        return [j for j in range(7) if self.state[0, j] == 0]  # Verificar si la columna no está llena

    def update(self, symbol, col):
        for row in range(5, -1, -1):  # Llenar desde la parte inferior
            if self.state[row, col] == 0:
                self.state[row, col] = symbol
                return row, col
        raise ValueError("Columna llena!")

    def is_game_over(self):
        # Comprobar filas, columnas y diagonales
        for row in range(6):
            for col in range(7):
                if col + 3 < 7 and np.all(self.state[row, col:col + 4] == self.state[row, col]) and self.state[row, col] != 0:
                    return self.state[row, col]
                if row + 3 < 6 and np.all(self.state[row:row + 4, col] == self.state[row, col]) and self.state[row, col] != 0:
                    return self.state[row, col]
                if col + 3 < 7 and row + 3 < 6 and np.all([self.state[row + i, col + i] == self.state[row, col] for i in range(4)]) and self.state[row, col] != 0:
                    return self.state[row, col]
                if col - 3 >= 0 and row + 3 < 6 and np.all([self.state[row + i, col - i] == self.state[row, col] for i in range(4)]) and self.state[row, col] != 0:
                    return self.state[row, col]
        # Empate
        if len(self.valid_moves()) == 0:
            return 0
        # Seguir jugando
        return None

    def reset(self):
        self.state = np.zeros((6, 7))

from tqdm import tqdm

In [5]:

class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    def selfplay(self, rounds=100):
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False
            while not game_over:
                for player in self.players:
                    action = player.move(self.board)
                    row, col = self.board.update(player.symbol, action)
                    for player in self.players:
                        player.update(self.board)
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward()
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1
        return wins

    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0:  # empate
            for player in self.players:
                player.reward(0.5)
        else:  # le damos 1 recompensa al jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)


In [6]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {}  # tabla con pares estado -> valor
        self.alpha = alpha         # learning rate
        self.positions = []       # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp   # probabilidad de explorar

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        # exploracion
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # vamos a una posición aleatoria
            return np.random.choice(valid_moves)
        # explotacion
        # vamos a la posición con más valor
        max_value = -1000
        for col in valid_moves:
            next_board = board.state.copy()
            for row in range(5, -1, -1):
                if next_board[row, col] == 0:
                    next_board[row, col] = self.symbol
                    break
            next_state = str(next_board.reshape(6 * 7))
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_col = col
        return best_col

    def update(self, board):
        self.positions.append(str(board.state.reshape(6 * 7)))

    def reward(self, reward):
        # al final de la partida (cuando recibimos la recompensa)
        # iteramos por todos los estados actualizando su valor en la tabla
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]


In [7]:
# Entrenar los agentes
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)
game.selfplay(10000)

100%|██████████| 10000/10000 [08:44<00:00, 19.06it/s]


[5581, 4364]

In [8]:
import pandas as pd

funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

print(tabla)

                                                   estado  valor
0       [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  ...    1.0
1       [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  ...    1.0
2       [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  ...    1.0
3       [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  ...    1.0
4       [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  ...    1.0
...                                                   ...    ...
147670  [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  ...    0.0
147671  [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  ...    0.0
147672  [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  ...    0.0
147673  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  ...    0.0
147674  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  ...    0.0

[147675 rows x 2 columns]


In [9]:
import pickle

with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)
